### Using the BISON API
The USGS provides an API for accessing species observation data. https://bison.usgs.gov/doc/api.jsp

This API is much better documented than the NWIS API and we'll use it to dig a bit deeper into how the `requests` package can faciliate data access via APIs. 

We'll begin by replicating the example API call they show on their web page:<br> 
[https://bison.usgs.gov/api/search.json?species=Bison bison&type=scientific_name&start=0&count=1](https://bison.usgs.gov/api/search.json?species=Bison bison&type=scientific_name&start=0&count=1)

In [ ]:
#First, import the wonderful requests module
import requests
import folium
import pandas as pd
import numpy as np

In [ ]:
#Now, we'll deconstruct the example URL into the service URL and parameters, saving the paramters as a dictionary
url = 'http://bison.usgs.gov/api/search.json'
params = {'species':'Bison bison',
          'type':'scientific_name',
          'start':'0',
          'count':'100'
         }

In [ ]:
#Submit the request
response = requests.get(url,params)

In [ ]:
data = response.json()

In [ ]:
import pandas as pd
df = pd.DataFrame(data['data'])
df.head()

And Pandas allows us to do some nifty analyses, including subsetting records where the provider is 'iNaturalist.org'

In [ ]:
df[df.provider == 'iNaturalist.org']

In [ ]:
df.dtypes

In [ ]:
#Find center coordinates from medians of lat and long columns
meanLat = df['decimalLatitude'].median()
meanLng = df['decimalLongitude'].median()

#Create the initial map
m = folium.Map(location=[meanLat,meanLng],
               zoom_start=3,
               tiles='stamenterrain')

#Loop through all features and add them to the map as markers
for row in df.itertuples():
    #Get info for the record
    lat = float(row.decimalLatitude)
    lng = float(row.decimalLongitude)
    name = row.provider
    #Create the marker object, adding them to the map object
    folium.CircleMarker(location=[lat,lng],
                        popup=name,
                        color='red',
                        fill=True,
                        fill_opacity=0.6,
                        radius=5.0,
                        stroke=False).add_to(m)
#Show the map
m